In [13]:
import requests
import xml.etree.ElementTree as ET

r = requests.get("http://178.62.245.17/air/airquality.php")


xmparsed = ET.fromstring(r.text)

In [404]:
import pandas as pd

data = []
for i in xmparsed[1]:
    row = []
    for j in i[:3]:
        row.append(j.text)
    data.append(row)

map_dict ={2: {'name': 'NO2', 'thresholds': [100, 150, 200]},
 4: {'name': 'SO2', 'thresholds': [150, 250, 350]},
 5: {'name': 'O3', 'thresholds': [100, 140, 180]},
 6: {'name': 'CO', 'thresholds': [7000, 15000, 20000]},
 25: {'name': 'PM10', 'thresholds': [50, 100, 200]},
 26: {'name': 'PM2.5', 'thresholds': [25, 50, 100]},
 45: {'name': 'C6H6', 'thresholds': [5, 10, 15]}}

def get_level(pol_id, value, mapings):
    thresh = mapings[pol_id]["thresholds"]
    
    lvl = 0
    
    for i in thresh:
        if value > i:
            
            lvl+=1

    return(lvl+1)

pollution_data = {}

for count, row in enumerate(data):
    if row[0] not in pollution_data:
        pollution_data[row[0]] = {}
    
    if int(row[1]) in map_dict:
#         print(row)
#         row = row.append(get_level(int(row[1]), float(row[2]), map_dict))
        pollution_data[row[0]][row[1]] =get_level(int(row[1]), float(row[2]), map_dict)

pollution_indicator = {}


for key, mappings in pollution_data.items():
    
    cumul = 0
    maximum = 0
    count = 0

    for d, value in mappings.items():
#         print(maximum)
        count +=1
        cumul += int(value)
#         print(value)
        maximum = max(maximum, value)

    indicator = maximum

    ar_mean = cumul/count
    
    indicator_2 = 0
    if ar_mean > float(3):
        indicator_2 = 4

    pollution_indicator[key] = max(indicator, indicator_2)


In [405]:
pollution_indicator
    

{'1': 1,
 '10': 2,
 '11': 1,
 '12': 1,
 '13': 1,
 '14': 2,
 '15': 2,
 '16': 1,
 '2': 2,
 '3': 1,
 '4': 1,
 '5': 2,
 '6': 1,
 '7': 1,
 '8': 2,
 '9': 4}

In [370]:
pollution_data

{'1': {'2': 1, '25': 1, '26': 1, '4': 1, '45': 1, '5': 1, '6': 1},
 '10': {'2': 1, '5': 2},
 '11': {'2': 1, '5': 1},
 '12': {'5': 1},
 '13': {'2': 1, '5': 1},
 '14': {'2': 1, '4': 1, '45': 1, '5': 2, '6': 1},
 '15': {'2': 1, '25': 1, '4': 1, '45': 1, '5': 2, '6': 1},
 '16': {'2': 1, '25': 1, '26': 1, '4': 1, '5': 1, '6': 1},
 '2': {'2': 1, '4': 1, '5': 2, '6': 1},
 '3': {'2': 1, '25': 1, '4': 1, '5': 1, '6': 1},
 '4': {'2': 1, '5': 1, '6': 1},
 '5': {'2': 1, '25': 1, '26': 1, '4': 1, '45': 1, '5': 2, '6': 1},
 '6': {'2': 1, '4': 1, '5': 1, '6': 1},
 '7': {'2': 1, '25': 1, '4': 1, '45': 1, '5': 1, '6': 1},
 '8': {'2': 1, '25': 1, '26': 1, '4': 1, '45': 1, '5': 2},
 '9': {'2': 1, '25': 1, '26': 1, '4': 1, '45': 1, '5': 2, '6': 4}}

In [43]:
keys_as_str = """NO Μονοξείδιο Αζώτου [μg/m³] (id=1)
NO2 Διοξείδιο Αζώτου [μg/m³] (id=2)
NOx Οξείδια του Αζώτου [μg/m³] (id=3)
SO2 Διοξείδιο Θείου [μg/m³] (id=4)
O3 Όζον [μg/m³] (id=5)
CO Μονοξείδιο 'Ανθρακα [μg/m³] (id=6)
PM10 Σωματίδια 10 μm [μg/m³] (id=25)
PM2.5 Σωματίδια 2.5 μm [μg/m³] (id=26)
C6H6 Βενζόλιο [μg/m³] (id=45)"""

In [49]:
outer_list_transform = keys_as_str.split("\n")

pollutants_map = {}
for i in outer_list_transform:
    as_array = i.split(" ")
    station_id = as_array[-1].replace("(id=", "").replace(")", "")
    pollutant_name = as_array[0]
    pollutants_map[station_id] = pollutant_name

In [52]:
# pd.DataFrame.from_dict(pollutants_map, orient="index").to_csv("pollutant_id_map.csv")

In [54]:
for key in pollutants_map:
    pollutants_map[key] = {pollutants_map[key], []}

{'1': 'NO',
 '2': 'NO2',
 '25': 'PM10',
 '26': 'PM2.5',
 '3': 'NOx',
 '4': 'SO2',
 '45': 'C6H6',
 '5': 'O3',
 '6': 'CO'}

In [58]:
sdf = pd.read_clipboard()

In [61]:
sdf.to_csv("pollutant_scale.csv")

In [77]:
# sdf = sdf.set_index("Pollutant")
for j, col in enumerate(sdf):
        if j<5:
        sdf[j] = sdf[col].apply(lambda x: str(x).split("-")[-1])

In [80]:
sdf = pd.DataFrame(sdf[[0,1,2,3,4]])

In [91]:
mod_pol = {k: pollutants_map[k] for k in pollutants_map if k!="1" }

In [103]:
mod_pol = {v:k for k, v in mod_pol.items()}
sdf["id"] = list(map(lambda x: mod_pol[x], sdf.index))

In [108]:
sdf["name"] = sdf.index
sdf = sdf.set_index("id")

In [123]:
sdf = pd.read_csv("thresholds.csv")

In [ ]:
def assign_pol_level(pollutant_id, level):
    
    if level<50:
        return 1
    if level<100:
        return 2
    if level<
    

In [124]:
sdf

,id,0,1,2,3,name
0,25,50,100,200,200,PM10
1,26,25,50,100,100,PM2.5
2,5,100,140,180,180,O3
3,2,100,150,200,200,NO2
4,4,150,250,350,350,SO2
5,6,7000,15000,20000,20000,CO
6,45,5,10,15,15,C6H6


In [180]:
map_dict = {}

for zaebalo in sdf.values:
    map_dict[zaebalo[0]] = {"name": zaebalo[-1]}
    map_dict[zaebalo[0]]["thresholds"] = [zaebalo[1],zaebalo[2],zaebalo[3]]
    

In [181]:
map_dict

{2: {'name': 'NO2', 'thresholds': [100, 150, 200]},
 4: {'name': 'SO2', 'thresholds': [150, 250, 350]},
 5: {'name': 'O3', 'thresholds': [100, 140, 180]},
 6: {'name': 'CO', 'thresholds': [7000, 15000, 20000]},
 25: {'name': 'PM10', 'thresholds': [50, 100, 200]},
 26: {'name': 'PM2.5', 'thresholds': [25, 50, 100]},
 45: {'name': 'C6H6', 'thresholds': [5, 10, 15]}}

In [197]:
import json

dumped = json.dumps(map_dict)

In [203]:
def get_level(pol_id, value, mapings):
    thresh = mapings[pol_id]["thresholds"]
    
    lvl = 0
    
    for i in thresh:
        if value > i:
            print(value, i)
            lvl+=1

    return(lvl+1)
print(get_level(45, 1, map_dict))   

1


False

In [294]:
df["check_int"] = df[1].apply(lambda x: int(x))

In [308]:
df.loc[df["check_int"]==2]

,0,1,2,3,4,5,6,7,8,9,check_int
5,1,2,22.651833,2018-09-15 13:00:00,Nicosia - Traffic Station,ÎÎµÏ ÎºÏÏÎ¯Î± - ÎÏ ÎºÎ»Î¿ÏÎ¿ÏÎ¹Î±ÎºÏÏ ...,Traffic,ÎÏ ÎºÎ»Î¿ÏÎ¿ÏÎ¹Î±ÎºÏÏ,35.15192246950294,33.347919957077806,2
23,2,2,6.3320300000000005,2018-09-15 13:00:00,Nicosia - Residential Station,ÎÎµÏ ÎºÏÏÎ¯Î± - ÎÎ¹ÎºÎ¹ÏÏÎ¹ÎºÏÏ Î£ÏÎ±...,Urban,ÎÎ¹ÎºÎ¹ÏÏÎ¹ÎºÏÏ,35.1269444,33.33166670000003,2
39,3,2,13.371870000000001,2018-09-15 13:00:00,Limassol - Traffic Station,ÎÎµÎ¼ÎµÏÏÏ - ÎÏ ÎºÎ»Î¿ÏÎ¿ÏÎ¹Î±ÎºÏÏ Î£...,Traffic,ÎÏ ÎºÎ»Î¿ÏÎ¿ÏÎ¹Î±ÎºÏÏ,34.6861111,33.03555559999995,2
51,3,2,12.797970000000001,2018-09-15 12:00:00,Limassol - Traffic Station,ÎÎµÎ¼ÎµÏÏÏ - ÎÏ ÎºÎ»Î¿ÏÎ¿ÏÎ¹Î±ÎºÏÏ Î£...,Traffic,ÎÏ ÎºÎ»Î¿ÏÎ¿ÏÎ¹Î±ÎºÏÏ,34.6861111,33.03555559999995,2
56,4,2,16.853530000000003,2017-06-28 08:00:00,Limassol - Residential Station,ÎÎµÎ¼ÎµÏÏÏ - ÎÎ¹ÎºÎ¹ÏÏÎ¹ÎºÏÏ Î£ÏÎ±Î¸...,Urban,ÎÎ¹ÎºÎ¹ÏÏÎ¹ÎºÏÏ,34.6816667,33.017777799999976,2
66,5,2,4.924062,2018-09-15 13:00:00,Larnaca - Traffic Station,ÎÎ¬ÏÎ½Î±ÎºÎ± - ÎÏ ÎºÎ»Î¿ÏÎ¿ÏÎ¹Î±ÎºÏÏ Î£...,Traffic,ÎÏ ÎºÎ»Î¿ÏÎ¿ÏÎ¹Î±ÎºÏÏ,34.9166667,33.627500000000055,2
80,6,2,12.12842,2016-11-18 14:00:00,Larnaca - Residential Station,ÎÎ¬ÏÎ½Î±ÎºÎ± - ÎÎ¹ÎºÎ¹ÏÏÎ¹ÎºÏÏ Î£ÏÎ±Î¸...,Urban,ÎÎ¹ÎºÎ¹ÏÏÎ¹ÎºÏÏ,34.9136111,33.61583329999996,2
86,7,2,12.45363,2016-02-12 09:00:00,Paphos - Traffic Station,Î Î¬ÏÎ¿Ï - ÎÏ ÎºÎ»Î¿ÏÎ¿ÏÎ¹Î±ÎºÏÏ Î£ÏÎ±...,Traffic,ÎÏ ÎºÎ»Î¿ÏÎ¿ÏÎ¹Î±ÎºÏÏ,34.7752778,32.42194440000003,2
99,8,2,3.667221,2018-09-15 13:00:00,Zygi - Industrial Station,ÎÏÎ³Î¹ - ÎÎ¹Î¿Î¼Î·ÏÎ±Î½Î¹ÎºÏÏ Î£ÏÎ±Î¸Î¼...,Industrial,ÎÎ¹Î¿Î¼Î·ÏÎ±Î½Î¹ÎºÏÏ,34.7294444,33.33749999999998,2
117,9,2,1.74083,2018-09-15 13:00:00,Ayia Marina Xyliatou - Background Station,ÎÎ³Î¯Î± ÎÎ±ÏÎ¯Î½Î± ÎÏ Î»Î¹Î¬ÏÎ¿Ï - Î£ÏÎ...,Rural,ÎÎ³ÏÎ¿ÏÎ¹ÎºÏÏ,35.0380556,33.05777779999994,2
